Import Libraries

In [1]:
import pandas as pd

Import CSV

In [ ]:
##select dataset here
df = pd.read_csv('D:/ML Projects/Data_Cleanup/data/raw/Highest_career_batting_average_in_Tests.csv')
df

In [3]:
df = df.rename(columns = {
                    
                    'Span':'span',
                    'Mat':'matches_played',
                    'Inns':'innings',
                    'NO':'not_out',
                    'Runs':'runs',
                    'HS':'highest_inns_score',
                    'Ave':'batting_average',
                    'BF':'balls_faced',
                    'SR':'strike_rate',
                    '100':'hundreds_scored',
                    '50':'fifties_scored',
                    '0':'ducks_scored',
                    '4s':'fours',
                    '6s':'sixes',
                    'Player':'player',
                    'Rank':'rank'

})

Check null and empty

In [ ]:
df.isna().sum()

In [ ]:
df.isnull().any()

Clean balls_faced and convert to int

In [ ]:
df[df['balls_faced'].isna() == 1]

In [7]:
df['balls_faced']=df['balls_faced'].str.split('+').str.get(0)

In [8]:
df['balls_faced'] = pd.to_numeric(df['balls_faced'], errors='coerce')

Check if there are outliers

In [ ]:
q1 = df['balls_faced'].quantile(0.25)
q3 = df['balls_faced'].quantile(0.75)
IQR = q3 - q1

lower_fence = q1 - 1.5 * IQR
upper_fence = q3 + 1.5 * IQR

outliers = df[(df['balls_faced'] < lower_fence) | (df['balls_faced'] > upper_fence)]

print(df['balls_faced'].min())
print(lower_fence)
print(df['balls_faced'].max())
print(upper_fence)

outliers.empty

Fill balls_faced empty values with the mean since there are no outliers of all values

In [10]:
df['balls_faced'].dtype


dtype('float64')

In [11]:
df['balls_faced'] = df['balls_faced'].fillna(df['balls_faced'].mean())


In [ ]:
df['balls_faced'] = df['balls_faced'].astype(int)
df


Do the same for strike_rate and ducks_scored

In [ ]:
df[(df['strike_rate'].isna() == 1) | (df['ducks_scored'].isna() == 1)]


In [ ]:
df.dtypes

In [ ]:
IQSR1 = df['strike_rate'].quantile(0.25)
IQSR3 = df['strike_rate'].quantile(0.75)
IQSRR = IQSR3 - IQSR1

IQDS1 = df['ducks_scored'].quantile(0.25)
IQDS3 = df['ducks_scored'].quantile(0.75)
IQDSR = IQDS3 - IQDS1

lower_fenceSR = IQSR1 - (1.5 * IQSRR)
upper_fenceSR = IQSR3 + (1.5 * IQSRR)
outlierSR = df[(df['strike_rate']<lower_fenceSR) | (df['strike_rate']>upper_fenceSR)]

lower_fenceDS = IQDS1 - (1.5 * IQDSR)
upper_fenceDS = IQDS3 + (1.5 * IQDSR)
outlierDS = df[(df['ducks_scored']<lower_fenceDS) | (df['ducks_scored']>upper_fenceDS)]

print(outlierSR.empty)
print(outlierDS.empty)


print(df['strike_rate'].min())
print(lower_fenceSR)
print(df['strike_rate'].max())
print(upper_fenceSR)


Dataset has outliers in strike_rate column but not in ducks_scored. so we'll use median for strike_rate and mean for ducks_scored.

In [16]:
df['strike_rate'] = df['strike_rate'].fillna(df['strike_rate'].median())
df['ducks_scored'] = df['ducks_scored'].fillna(df['ducks_scored'].mean())

In [17]:
df[(df['strike_rate'].isna() == 1) | (df['ducks_scored'].isna() == 1)]

,rank,player,span,matches_played,innings,not_out,runs,highest_inns_score,batting_average,balls_faced,strike_rate,hundreds_scored,fifties_scored,ducks_scored,fours,sixes


Check for duplicates

In [ ]:
df.duplicated()

Split span to start and end and drop span afterward

In [19]:
df['rookie_year'] = df['span'].str.split(pat = '-').str.get(0)
df['final_year'] = df['span'].str.split(pat = '-').str.get(1)
df = df.drop(['span'], axis = 1)


In [ ]:
df

Split Country from player name

In [21]:
df['country'] = df['player'].str.split('(').str.get(1)
df['country'].str.split(')').str.get(0)
df['country'] = df['country'].str.split(')').str.get(0)

df['player'] = df['player'].str.split('(').str.get(0)


In [ ]:
df.dtypes

Correct data types

In [23]:
df['highest_inns_score'] = df['highest_inns_score'].str.split(pat = '*').str.get(0)


In [24]:
df['highest_inns_score'] = pd.to_numeric(df['highest_inns_score'], errors='coerce')

In [25]:
df['highest_inns_score'].dtypes

dtype('int64')

In [26]:
cols = ['rookie_year', 'final_year']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [27]:
cols = ['player', 'country']
df[cols] = df[cols].astype('string')

In [28]:
df['fours']=df['fours'].str.split('+').str.get(0)
df['sixes']=df['sixes'].str.split('+').str.get(0)

In [29]:
cols = ['fours' , 'sixes']
df[cols] = df[cols].apply(pd.to_numeric, errors = 'coerce')

In [30]:
df['ducks_scored'] = df['ducks_scored'].astype(int)

In [31]:
df = df.drop(['rank'], axis = 1)

Save the processed CSV

In [ ]:
##change the output directory accordingly. save the processed csv in data/processed
df.to_csv("D:/ML Projects/Data_Cleanup/data/processed/Processed.csv", index=False)

In [ ]:
df

Visualization

Player grouping by country

In [ ]:
import plotly.express as px

fig = px.treemap(
    df, 
    path=['country', 'player'], 
    values='runs',               
    color='runs',                              
    color_continuous_scale='RdBu',
    title="Player grouping by country sorted by runs"
)
fig.show()
fig.write_image("D:/ML Projects/Data_Cleanup/plots/1.png")

Gantt Chart of player career timeline

In [ ]:

df['start_date'] = pd.to_datetime(df['rookie_year'].astype(str) + '-01-01')
df['end_date'] = pd.to_datetime(df['final_year'].astype(str) + '-12-31')


fig = px.timeline(
    df, 
    x_start="start_date", 
    x_end="end_date", 
    y="player", 
    color="matches_played",
    title="Player Career Timelines"
)


fig.update_yaxes(categoryorder="total ascending") 

fig.show()
fig.write_image("D:/ML Projects/Data_Cleanup/plots/2.png")

Bar chart of how many runs were from running, 4s and 6s

In [ ]:
import plotly.express as px


df['runs_from_4s'] = df['fours'] * 4
df['runs_from_6s'] = df['sixes'] * 6
df['runs_running'] = df['runs'] - (df['runs_from_4s'] + df['runs_from_6s'])


df_boundary = df.melt(
    id_vars=['player'], 
    value_vars=['runs_running', 'runs_from_4s', 'runs_from_6s'],
    var_name='Run Type', 
    value_name='Run Amount'
)


fig = px.bar(
    df_boundary, 
    x="player", 
    y="Run Amount", 
    color="Run Type", 
    title="Run Composition: Running vs. Boundaries",
    
    color_discrete_map={
        'runs_running': "#19191F", 
        'runs_from_4s': '#636EFA', 
        'runs_from_6s': '#EF553B'
    }
)

fig.show()
fig.write_image("D:/ML Projects/Data_Cleanup/plots/3.png")